[link text](https://colab.research.google.com/drive/1BypgxIC1qltpS5pdcFvWUvxxYhuJkLFW?usp=sharing)

In [ ]:
pip install fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 40.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import fitz  # PyMuPDF

# Read the CSV file
df = pd.read_csv('Onion.csv')

# Define the table pattern
table_pattern = r"(?:Red Onion \(Local\)|Red Onion \(lmp\)|Big Onion \(Local\)|Big Onion \(Imp\))\s+Rs\./kg\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)"

# Function to extract text from the PDF
def extract_text_from_pdf(pdf_url):
    response = requests.get(pdf_url, stream=True)

    # Check if the PDF is empty
    if 'Content-Length' in response.headers and int(response.headers['Content-Length']) == 0:
        print(f"Empty document for {pdf_url}. Skipping...")
        return None

    with open('temp.pdf', 'wb') as pdf_file:
        pdf_file.write(response.content)

    doc = fitz.open('temp.pdf')
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()

    return text

# Iterate through rows in the dataframe
prices_data = {'Date': [], 'Onion Type': []}
for j in range(1, 9):
    prices_data[f'Price{j}'] = []

for index, row in df.iterrows():
    date = row['Date']
    pdf_url = row['links-href']

    # Extract text from the PDF
    pdf_text = extract_text_from_pdf(pdf_url)

    if pdf_text is not None:
        # Search for the table with prices in the text
        table_match = re.findall(table_pattern, pdf_text)

        # Check if table matches are found
        if table_match:
            onion_types = ["Red Onion (Local)", "Red Onion (lmp)", "Big Onion (Local)", "Big Onion (Imp)"]

            for i, onion_type in enumerate(onion_types):
                prices_data['Date'].append(date)
                prices_data['Onion Type'].append(onion_type)

                for j in range(8):
                    prices_data[f'Price{j + 1}'].append(table_match[i][j])
        else:
            print(f"No table pattern found in the PDF for {date}.")

# Create a DataFrame from the collected data
prices_df = pd.DataFrame(prices_data)

# Save the results to a new CSV file
prices_df.to_csv('prices_data.csv', index=False)

# Remove temporary PDF file
import os
os.remove('temp.pdf')


Empty document for https://www.cbsl.gov.lk/sites/default/files/cbslweb_documents/statistics/pricerpt/price_report_20220914.pdf. Skipping...
No table pattern found in the PDF for 3/16/2022.


In [ ]:
import pandas as pd

# Load the prices_data.csv file
df = pd.read_csv('prices_data.csv')

# Drop unnecessary columns (Price1, Price3, Price5, Price7)
df = df.drop(columns=['Price1', 'Price3', 'Price5', 'Price7'])

# Rename columns
column_mapping = {
    'Price2': 'Wholesale_Pettah',
    'Price4': 'Wholesale_Dambulla',
    'Price6': 'Retail_Pettah',
    'Price8': 'Retail_Dambulla'
}

df = df.rename(columns=column_mapping)

# Replace 'n.a.' with NaN
df.replace('n.a.', pd.NA, inplace=True)

# Convert columns to numeric (if not already) to handle NaN values
numeric_columns = ['Wholesale_Pettah', 'Wholesale_Dambulla', 'Retail_Pettah', 'Retail_Dambulla']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Sort DataFrame by 'Date' and 'Onion Type'
df = df.sort_values(['Onion Type', 'Date'])

# Forward fill missing values within each group of 'Onion Type'
df = df.groupby('Onion Type').apply(lambda group: group.ffill())

# Backward fill remaining missing values within each group of 'Onion Type'
df = df.groupby('Onion Type').apply(lambda group: group.bfill())

# Save the cleaned DataFrame to a new CSV file
df.to_csv('prices_data_filled.csv', index=False)


<ipython-input-28-3cedf43c255a>:33: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Onion Type').apply(lambda group: group.ffill())
<ipython-input-28-3cedf43c255a>:36: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Onion Type').apply(lambda group: group.bfill())


In [ ]:
df_cleaned = pd.read_csv('prices_data_filled.csv')
df_cleaned.head()

,Date,Onion Type,Wholesale_Pettah,Wholesale_Dambulla,Retail_Pettah,Retail_Dambulla
0,2021-01-01,Big Onion (Imp),79.0,88.0,130.0,98.0
1,2021-01-04,Big Onion (Imp),72.0,73.0,130.0,83.0
2,2021-01-05,Big Onion (Imp),69.0,88.0,130.0,98.0
3,2021-01-06,Big Onion (Imp),73.0,53.0,130.0,63.0
4,2021-01-07,Big Onion (Imp),63.0,98.0,130.0,108.0


In [ ]:
# Count NaN values
nan_count = df_cleaned.isna().sum()

# Print the count of NaN values
print("NaN Values Count:")
print(nan_count)

NaN Values Count:
Date                  0
Onion Type            0
Wholesale_Pettah      0
Wholesale_Dambulla    0
Retail_Pettah         0
Retail_Dambulla       0
dtype: int64


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('prices_data_filled.csv')

# Create separate data frames for each onion type
red_onion_local_df = df[df['Onion Type'] == 'Red Onion (Local)']
red_onion_imp_df = df[df['Onion Type'] == 'Red Onion (lmp)']
big_onion_local_df = df[df['Onion Type'] == 'Big Onion (Local)']
big_onion_imp_df = df[df['Onion Type'] == 'Big Onion (Imp)']

# Print or do further operations with the data frames
print("Red Onion (Local) Data Frame:")
print(red_onion_local_df)


Red Onion (Local) Data Frame:
            Date         Onion Type  Wholesale_Pettah  Wholesale_Dambulla  \
1420  2021-01-01  Red Onion (Local)             600.0               475.0   
1421  2021-01-04  Red Onion (Local)             600.0               450.0   
1422  2021-01-05  Red Onion (Local)             600.0               470.0   
1423  2021-01-06  Red Onion (Local)             600.0               380.0   
1424  2021-01-07  Red Onion (Local)             600.0               300.0   
...          ...                ...               ...                 ...   
2125  2023-12-18  Red Onion (Local)             450.0               365.0   
2126  2023-12-19  Red Onion (Local)             500.0               375.0   
2127  2023-12-20  Red Onion (Local)             500.0               395.0   
2128  2023-12-21  Red Onion (Local)             500.0               300.0   
2129  2023-12-22  Red Onion (Local)             500.0               315.0   

      Retail_Pettah  Retail_Dambulla  
1420  

In [ ]:
print("\nRed Onion (Imp) Data Frame:")
print(red_onion_imp_df)




Red Onion (Imp) Data Frame:
            Date       Onion Type  Wholesale_Pettah  Wholesale_Dambulla  \
2130  2021-01-01  Red Onion (lmp)             600.0               398.0   
2131  2021-01-04  Red Onion (lmp)             600.0               398.0   
2132  2021-01-05  Red Onion (lmp)             525.0               398.0   
2133  2021-01-06  Red Onion (lmp)             495.0               398.0   
2134  2021-01-07  Red Onion (lmp)             395.0               398.0   
...          ...              ...               ...                 ...   
2835  2023-12-18  Red Onion (lmp)             410.0               375.0   
2836  2023-12-19  Red Onion (lmp)             415.0               445.0   
2837  2023-12-20  Red Onion (lmp)             407.0               435.0   
2838  2023-12-21  Red Onion (lmp)             420.0               445.0   
2839  2023-12-22  Red Onion (lmp)             410.0               410.0   

      Retail_Pettah  Retail_Dambulla  
2130          480.0            

In [ ]:
print("\nBig Onion (Local) Data Frame:")
print(big_onion_local_df)



Big Onion (Local) Data Frame:
            Date         Onion Type  Wholesale_Pettah  Wholesale_Dambulla  \
710   2021-01-01  Big Onion (Local)             145.0                98.0   
711   2021-01-04  Big Onion (Local)             145.0                98.0   
712   2021-01-05  Big Onion (Local)             145.0                98.0   
713   2021-01-06  Big Onion (Local)             145.0                98.0   
714   2021-01-07  Big Onion (Local)             145.0                98.0   
...          ...                ...               ...                 ...   
1415  2023-12-18  Big Onion (Local)             240.0               365.0   
1416  2023-12-19  Big Onion (Local)             240.0               365.0   
1417  2023-12-20  Big Onion (Local)             240.0               365.0   
1418  2023-12-21  Big Onion (Local)             240.0               515.0   
1419  2023-12-22  Big Onion (Local)             240.0               515.0   

      Retail_Pettah  Retail_Dambulla  
710  

In [ ]:

print("\nBig Onion (Imp) Data Frame:")
print(big_onion_imp_df)



Big Onion (Imp) Data Frame:
           Date       Onion Type  Wholesale_Pettah  Wholesale_Dambulla  \
0    2021-01-01  Big Onion (Imp)              79.0                88.0   
1    2021-01-04  Big Onion (Imp)              72.0                73.0   
2    2021-01-05  Big Onion (Imp)              69.0                88.0   
3    2021-01-06  Big Onion (Imp)              73.0                53.0   
4    2021-01-07  Big Onion (Imp)              63.0                98.0   
..          ...              ...               ...                 ...   
705  2023-12-18  Big Onion (Imp)             428.0               415.0   
706  2023-12-19  Big Onion (Imp)             483.0               390.0   
707  2023-12-20  Big Onion (Imp)             492.0               425.0   
708  2023-12-21  Big Onion (Imp)             475.0               435.0   
709  2023-12-22  Big Onion (Imp)             418.0               425.0   

     Retail_Pettah  Retail_Dambulla  
0            130.0             98.0  
1     